In [3]:
import torch
from model import load_resnet_model
from dataloader import create_dataloader, read_tfrecord
from train import train_model
from checkpoint import save_checkpoint, load_checkpoint
from tqdm import tqdm

In [5]:
def concatenate_and_save_tfrecords(tfrecord_paths):
    all_data = []
    all_labels = []
    count = 0
    batch_number = 1
    for path in tqdm(tfrecord_paths):
        data, label = read_tfrecord(path)  # Read each TFRecord file
        all_data.append(data)
        all_labels.append(label)
        count = count+1
        if count == 1024:
            # Concatenate all data and labels
            concatenated_data = torch.cat(all_data, dim=0)
            concatenated_labels = torch.cat(all_labels, dim=0)
            all_data = []
            all_labels = []
            count = 0
            torch.save({'data': concatenated_data, 'labels': concatenated_labels}, f'data_batch_{batch_number}.pth')
            batch_number = batch_number + 1

    # Concatenate all data and labels
    concatenated_data = torch.cat(all_data, dim=0)
    concatenated_labels = torch.cat(all_labels, dim=0)

    # Save using torch.save or h5py
    torch.save({'data': concatenated_data, 'labels': concatenated_labels}, f'data_batch_{batch_number}.pth')

concatenate_and_save_tfrecords(file_paths)

100%|██████████| 2000/2000 [06:06<00:00,  5.46it/s]


In [ ]:
concatenated_dataset = ConcatenatedDataset('concatenated_data.pth')
concatenated_dataloader = DataLoader(concatenated_dataset, batch_size=32, shuffle=True)

In [5]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class ConcatenatedDataset(Dataset):
    def __init__(self, data_file):
        self.data = torch.load(data_file)['data']
        self.labels = torch.load(data_file)['labels']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

concatenated_dataset = ConcatenatedDataset('concatenated_data.pth')
concatenated_dataloader = DataLoader(concatenated_dataset, batch_size=32, shuffle=True)


In [19]:
model = load_resnet_model('resnet50', num_classes=10)

c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
model(a[0])

tensor([[ 0.0356,  0.3611, -0.0887, -0.1209,  0.1997, -0.0277, -0.6700, -0.0594,
         -0.1655, -0.3548]], grad_fn=<AddmmBackward0>)